In [1]:
! nvidia-smi

Tue May 17 23:16:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 N/A |                  N/A |
| 40%   49C    P0    N/A /  N/A |    315MiB /  2000MiB |     N/A      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:06:00.0 Off |                  N/A |
| 27%   

### Train MLM

In [2]:
model_path = '../input/deberta-v3-small'
model_name = model_path.split('/')[-1]

In [3]:
from transformers import DebertaV2Tokenizer, DebertaV2ForMaskedLM
import torch

In [4]:
tokenizer = DebertaV2Tokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = DebertaV2ForMaskedLM.from_pretrained(model_path)

Some weights of the model checkpoint at ../input/deberta-v3-small were not used when initializing DebertaV2ForMaskedLM: ['lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

In [6]:
print(model)

DebertaV2ForMaskedLM(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0): DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
              (

In [7]:
inputs = tokenizer("I love [MASK].", return_tensors="pt")

for k in inputs.keys():
  print(k, ' - ', inputs[k])

input_ids  -  tensor([[     1,    273,    472, 128000,    323,      2]])
token_type_ids  -  tensor([[0, 0, 0, 0, 0, 0]])
attention_mask  -  tensor([[1, 1, 1, 1, 1, 1]])


In [8]:
with torch.no_grad():
    logits = model(**inputs).logits

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

In [9]:
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

'graduating'

In [10]:
import transformers
from datasets import load_dataset
print(transformers.__version__)

4.18.0


In [11]:
datasets = load_dataset("text", data_files={
    "train": '../input/patent_data/train_v2.txt', 
    "validation": '../input/patent_data/valid.txt'
})

Using custom data configuration default-3a45591c4a018f2a


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
datasets["train"][10]

{'text': '&lsqb;0011&rsqb; Fluorescence polarization allows for development of homogenous screening assays, whereby it is not necessary to separate the products from the reaction between the biological compound and the fluorescent substance and the unreacted fluorescent substance. The method of the present invention is a high throughput assay that uses fluorescence polarization to identify inhibitors of nucleic acid ligases and helicases in a panel or library of test compounds, and is based on the comparison of the fluorescence polarization signals.'}

In [13]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(datasets["train"])

,text
0,"&lsqb;0065&rsqb; Various alternative structures are possible in accordance with the inventive concept. For example, it is conceivable for both a glazing leg (to which the glazing pane is mounted) and a glazing bead (which is folded against the glazing pane after the pane is mounted to the glazing leg) to be formed in a unitary hinged structure (or as separate hinged components). In such a structure, the glazing leg would first be folded into place along a corresponding hinge line. The glazing pane would be mounted to the glazing leg. Then, the bead would be folded along its hinge line against the outer surface of the glazing pane."
1,A320
2,"so arranged that, as a tensioning load is increasingly applied to the second end portion of the shank, relative axial movement between the inner and outer sleeves will cause the outer sleeve to expand diametrically into engagement with the at least two components up to a preset limit, at which the compression ring will allow the shank to move relative to the inner and outer sleeves and to the at least two components."
3,The substrate 110 may be a crystalline semiconductor formed of single crystal silicon.
4,"Then, when removing the operation to working handle 5, as shown in figure 11, stamping knife 92 is from rotation posture 92(R) return to perforation posture 92(H), stamping knife 92 rises with the applying power of incision cutter 91 by helical spring S4, extracts from paper using P.Now, the raised piece P11 inserting the window 911 of incision cutter 91 is pulled out the one side Pa side of paper using P, by this incision cutter 91 in conjunction with multiple paper using P through cut hole P2.Like this, when extracting stamping knife 92 and incision cutter 91 from paper using P, dropped to the body 3 of extreme lower position by load during stamping-out, by the applying power of helical spring S1, rise to initial floating position, paper using P leaves from pressed sheet 22.If extract paper using P out from binder 1 in this condition, then in this way, as shown in figure 12, by being arranged on the outer peripheral face of the above-mentioned compressing roller 8 of above-mentioned body 3 and above-mentioned paper using bearing plane 61, make the bonding part P3 of above-mentioned paper using P in thickness direction compression, make above-mentioned book B.In addition, when above-mentioned stamping knife 92 and above-mentioned incision cutter 91 can not be extracted the one side Pa side of paper using P by the elastic force by means of only above-mentioned helical spring S4, as long as operate above-mentioned working handle 5 upward, this is helped to extract."
5,"The application and action of the oil additives can be further improved by combining them with surface-active substances such as nonionic, anionic or cationic surfactants. Examples of suitable anionic, nonionic and cationic surfactants are listed on pages 7 and 8 of WO 97/34485. Preferred surfactants are anionic surfactants of the dodecyl-benzylsulfonate type, in particular of their calcium salts, and also nonionic surfactants of the aliphatic alcohol ethoxylate type. Particularly preferred ethoxylated C 12 -C 22 fatty alcohols having a degree of ethoxylation of 5-40. An example of a commercially available surfactant is the Genapol type (Clariant AG). Also preferred are silicone surfactants, particularly, for example, Silwet L-77®, and polyalkyl-oxide-modified heptamethyltrisiloxane, which is commercially available as a perfluorinated surfactant. The concentration of the surfactant relative to the total amount of additives is generally between 1 and 30% by weight. Examples of oil additives composed of a mixture of an oil or a mineral oil or a derivative thereof and a surfactant include Edenor ME SU®, Turbocharge® (Syngenta AG, CH) and Actipron® ( BP Oil UK Limited, GB)."
6,"(D3) In some embodiments of the method of D1 or D2, the method further comprises: prior to the step of detecting the swipe gesture, detecting

In [15]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=12, remove_columns=["text"])

#2:   0%|          | 0/97 [00:00<?, ?ba/s]

#0:   0%|          | 0/97 [00:00<?, ?ba/s]

#1:   0%|          | 0/97 [00:00<?, ?ba/s]

#4:   0%|          | 0/97 [00:00<?, ?ba/s]

#3:   0%|          | 0/97 [00:00<?, ?ba/s]

...

#8:   0%|          | 0/97 [00:00<?, ?ba/s]

#5:   0%|          | 0/97 [00:00<?, ?ba/s]

#10:   0%|          | 0/97 [00:00<?, ?ba/s]

#9:   0%|          | 0/97 [00:00<?, ?ba/s]

#11:   0%|          | 0/97 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets["train"][1]

In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=12,
)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    f"{model_name}+finetuned-patent",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    push_to_hub=False,
    report_to=None,
    num_train_epochs=1, 
    save_total_limit=2,
    save_steps=100000, 
#    ignore_data_skip=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
import math
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")